# Ollie - Web Scraper

In [ ]:
import os, re, csv, json, time, random, logging, unicodedata, sys
from urllib.parse import urljoin, urlencode, urlparse
import requests
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter, Retry
import pandas as pd

sys.path.append(os.path.abspath(".."))

from skin import (
    SKIN_TYPE_CANONICAL_ORDER,
    SKIN_TYPE_SYNONYMS_PT,
)

from exclude import (
    EXCLUDE_KEYWORDS,
)

from ingredient import (
    INGREDIENTES_VALIDOS,
)

from benefits import (
    BENEFIT_SYNONYMS_PT,
    BENEFIT_CANONICAL_ORDER,
)



logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s")
logger = logging.getLogger("ollie")

In [7]:
BASE_URL = "https://meuollie.com.br"
COLLECTION_PATH = "/collections/loja-produtos-ollie"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
    "Accept-Language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
}



In [8]:

def make_session(max_retries=3, backoff=0.5, timeout=20):
    s = requests.Session()
    retry = Retry(
        total=max_retries, read=max_retries, connect=max_retries,
        backoff_factor=backoff, status_forcelist=[429,500,502,503,504],
        allowed_methods=frozenset(["GET"]), raise_on_status=False
    )
    adapter = HTTPAdapter(max_retries=retry, pool_connections=10, pool_maxsize=10)
    s.mount("http://", adapter)
    s.mount("https://", adapter)
    s.headers.update(HEADERS)
    s.timeout = timeout
    return s

def fetch_html(session, url, delay_range=(0.6,1.1)):
    time.sleep(random.uniform(*delay_range))
    r = session.get(url, timeout=session.timeout)
    if r.status_code >= 400:
        logger.warning("HTTP %s em %s", r.status_code, url)
    return r.text, r.status_code

def norm_space(t):
    if not t: return ""
    return re.sub(r"\s+", " ", t).strip()

def strip_accents(s: str) -> str:
    if not s: return ""
    return "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn")

def norm_text_for_match(t: str) -> str:
    t = t or ""
    t = strip_accents(t.lower())
    t = re.sub(r"\s+", " ", t)
    return t

def norm_price(t):
    if not t: return ""
    t = t.replace("R$", "").strip().replace(".","").replace(",",".")
    m = re.findall(r"[0-9.]+", t)
    return m[0] if m else ""

def slugify(text):
    t = text.lower().strip()
    t = re.sub(r"[^a-z0-9\-\_\sáàâãäéèêëíìîïóòôõöúùûüç]", "", t)
    t = t.replace(" ", "-")
    repl = (("á","a"),("à","a"),("â","a"),("ã","a"),("ä","a"),
            ("é","e"),("è","e"),("ê","e"),("ë","e"),
            ("í","i"),("ì","i"),("î","i"),("ï","i"),
            ("ó","o"),("ò","o"),("ô","o"),("õ","o"),("ö","o"),
            ("ú","u"),("ù","u"),("û","u"),("ü","u"),
            ("ç","c"))
    for a,b in repl: t = t.replace(a,b)
    t = re.sub(r"-+", "-", t)
    return t.strip("-")

def get_image_filename(url):
    if not url:
        return ""
    parsed = urlparse(url)
    filename = os.path.basename(parsed.path)
    return filename.lower() if filename else ""

def build_listing_url(page=1, phcursor=None):
    base = urljoin(BASE_URL, COLLECTION_PATH)
    params = []
    if page and page > 1:
        params.append(("page", str(page)))
    if phcursor:
        params.append(("phcursor", phcursor))
    return base + ("?" + urlencode(params) if params else "")

In [9]:
def parse_listing(html):
    soup = BeautifulSoup(html, "html.parser")
    product_urls = []
    for a in soup.select("a[href*='/products/']"):
        href = a.get("href") or ""
        if "/products/" in href:
            product_urls.append(urljoin(BASE_URL, href))

    next_url = None
    a_next = soup.find("a", attrs={"title": lambda x: x and "Avançar" in x})
    if a_next and a_next.get("href"):
        next_url = urljoin(BASE_URL, a_next["href"])
    if not next_url:
        for a in soup.select("a[href*='?page=']"):
            next_url = urljoin(BASE_URL, a.get("href"))
            break

    seen, uniq = set(), []
    for u in product_urls:
        if u not in seen:
            seen.add(u); uniq.append(u)
    return uniq, next_url

def _collect_section_text(soup, anchors=("PRINCIPAIS BENEFÍCIOS","BENEFÍCIOS","BENEFICIOS","RESULTADOS","POR QUE AMAR")):
    """
    Varre o HTML procurando blocos com títulos relacionados a benefícios
    e concatena o texto logo após esses títulos. Fallback: texto inteiro.
    """
    chunks = []
    big_text = norm_space(soup.get_text(" "))
    for b in soup.find_all(["b","strong","h1","h2","h3"]):
        title = norm_space(b.get_text()).upper()
        if any(a in title for a in anchors):
            # Coletar alguns irmãos/seguidores próximos
            cur = b.parent if b.parent else b
            hops, acc = 0, []
            while cur and hops < 12:
                cur = cur.find_next_sibling()
                if not cur: break
                if cur.name in ("p","div","span","ul","ol","li"):
                    acc.append(norm_space(cur.get_text(" ")))
                elif cur.name in ("h1","h2","h3","strong","b"):
                    break
                hops += 1
            if acc:
                chunks.append(" ".join(acc))
    if chunks:
        return " ".join(chunks)
    return big_text

In [10]:
def extract_benefits(soup):
    """
    Classifica benefícios do produto em rótulos canônicos com base em BENEFIT_SYNONYMS_PT.
    Retorna rótulos separados por vírgula, ordenados por BENEFIT_CANONICAL_ORDER.
    """
    text = _collect_section_text(soup)
    text_norm = norm_text_for_match(text)

    found = set()
    # percorrer dicionário de sinônimos -> rótulo canônico
    for canonical, synonyms in BENEFIT_SYNONYMS_PT.items():
        for syn in synonyms:
            syn_norm = norm_text_for_match(syn)
            if syn_norm and syn_norm in text_norm:
                found.add(canonical)
                break

    if not found:
        # fallback: cortar uma amostra do texto bruto (útil p/ inspeção)
        snippet = norm_space(text)[:220]
        return snippet

    # Ordenar pelo canônico definido
    ordered = [b for b in BENEFIT_CANONICAL_ORDER if b in found]
    return ", ".join(ordered)

def extract_skin_types(soup):
    """
    Extrai e classifica tipos/condições de pele com base em SKIN_TYPE_SYNONYMS_PT.
    Procura seção específica; fallback: corpo inteiro.
    """
    # Tentar achar âncoras típicas
    anchors = ("PARA QUAIS TIPOS DE PELE","TIPO DE PELE","TIPOS DE PELE","PELE")
    full_text = ""
    for b in soup.find_all(["b","strong","h1","h2","h3"]):
        t = norm_space(b.get_text()).upper()
        if any(a in t for a in anchors):
            cur = b.parent if b.parent else b
            acc, hops = [], 0
            while cur and hops < 10:
                cur = cur.find_next_sibling()
                if not cur: break
                if cur.name in ("p","div","span","ul","ol","li"):
                    acc.append(norm_space(cur.get_text(" ")))
                elif cur.name in ("h1","h2","h3","strong","b"):
                    break
                hops += 1
            if acc:
                full_text = " ".join(acc)
                break

    if not full_text:
        full_text = norm_space(soup.get_text(" "))

    txt = norm_text_for_match(full_text)
    found = set()
    # mapeia sinônimos -> rótulos
    for canonical, synonyms in SKIN_TYPE_SYNONYMS_PT.items():
        for syn in synonyms:
            syn_norm = norm_text_for_match(syn)
            if syn_norm and syn_norm in txt:
                found.add(canonical)
                break

    if found:
        ordered = [c for c in SKIN_TYPE_CANONICAL_ORDER if c in found]
        return ", ".join(ordered)
    else:
        # fallback: primeira frase/trecho
        return full_text[:200] + ("..." if len(full_text) > 200 else "")

def extract_active_ingredients(soup):
    """
    Extrai ingredientes ativos e filtra por INGREDIENTES_VALIDOS (case/acentos-insensitive).
    Tenta seções 'Principais ativos' / 'Composição'; fallback: texto inteiro.
    """
    # Seção "Principais ativos"
    text = ""
    for b in soup.find_all(["b","strong","h1","h2","h3"]):
        title = norm_space(b.get_text()).upper()
        if "PRINCIPAIS ATIVOS" in title or "ATIVOS" in title:
            cur = b.parent if b.parent else b
            hops, acc = 0, []
            while cur and hops < 12:
                cur = cur.find_next_sibling()
                if not cur: break
                if cur.name in ("p","div","span","ul","ol","li"):
                    acc.append(norm_space(cur.get_text(" ")))
                elif cur.name in ("h1","h2","h3","strong","b"):
                    break
                hops += 1
            if acc:
                text = " ".join(acc)
                break

    # Seção "Composição"
    if not text:
        for b in soup.find_all(["b","strong","h1","h2","h3"]):
            title = norm_space(b.get_text()).upper()
            if "COMPOSIÇÃO" in title or "COMPOSICAO" in title or "INGREDIENTES" in title:
                p = b.find_next("p")
                if p:
                    text = norm_space(p.get_text(" "))
                break

    if not text:
        text = norm_space(soup.get_text(" "))

    text_norm = norm_text_for_match(text)
    found = set()
    for ing in INGREDIENTES_VALIDOS:
        if norm_text_for_match(ing) in text_norm:
            found.add(ing)

    if found:
        return ", ".join(sorted(found, key=lambda x: strip_accents(x).lower()))
    return text[:300] + ("..." if len(text) > 300 else "")


def extract_image_best(soup):
    og = soup.select_one('meta[property="og:image"]')
    if og and og.get("content"):
        url = og["content"]
        if url.startswith("//"): url = "https:" + url
        return url

    best_url, best_w = "", -1
    for img in soup.select("img"):
        srcset = img.get("srcset") or ""
        src = img.get("src") or ""

        def absurl(u):
            if not u: return ""
            if u.startswith("//"): return "https:" + u
            return urljoin(BASE_URL, u)

        if srcset:
            for part in srcset.split(","):
                bits = part.strip().split()
                if not bits: 
                    continue
                cand, w = bits[0], -1
                if len(bits) > 1 and bits[1].endswith("w"):
                    try: w = int(bits[1][:-1])
                    except: w = -1
                if w > best_w:
                    best_w = w
                    best_url = absurl(cand)
        elif src:
            if best_w < 0:
                best_url = absurl(src)
                best_w = 0

    if best_url:
        if best_url.startswith("//"):
            best_url = "https:" + best_url
        return best_url
    return ""


In [11]:

# ====== PRODUTO ======
def parse_product(html, url):
    soup = BeautifulSoup(html, "html.parser")

    # Nome
    nome = ""
    name = soup.select_one("h1.h2.product-single__title") or soup.select_one("h1.product-single__title")
    if name: nome = norm_space(name.get_text())

    # Excluir termos indesejados
    if any(k in (nome or "").lower() for k in EXCLUDE_KEYWORDS):
        return None

    # Preço
    preco = ""
    price = soup.select_one("span.product__price") or soup.select_one("span[data-product-price]")
    if price: preco = norm_price(price.get_text())

    # Tipos de pele
    tipos_de_pele = extract_skin_types(soup)

    # Ingredientes
    ingredientes = extract_active_ingredients(soup)

    # Benefícios (NOVIDADE)
    beneficios = extract_benefits(soup)

    # Imagem
    imagem_url = extract_image_best(soup)
    imagem_filename = get_image_filename(imagem_url)

    data = {
        "site": "ollie",
        "categoria_textura": "",
        "nome": nome,
        "subtitulo": "",
        "preco": preco,
        "beneficios": beneficios,          # <-- preenchido
        "ingredientes": ingredientes,
        "tamanho": "",
        "tipos_de_pele": tipos_de_pele,
        "imagem": imagem_filename,
        "_imagem_url": imagem_url,
        "_url_produto": url
    }
    return data

# ====== DOWNLOAD IMG ======
def download_image(session, url, dest_dir, slug):
    if not url: return ""
    os.makedirs(dest_dir, exist_ok=True)
    m = re.search(r"[?&]width=(\d+)", url)
    width = m.group(1) if m else ""
    fname = f"{slug}__{width}.jpg" if width else f"{slug}.jpg"
    path = os.path.join(dest_dir, fname)
    try:
        r = session.get(url, timeout=session.timeout, headers=HEADERS)
        if r.status_code == 200:
            with open(path, "wb") as f: f.write(r.content)
            return path
        logging.warning("Falha ao baixar imagem %s (HTTP %s)", url, r.status_code)
    except Exception as e:
        logging.warning("Erro ao baixar imagem %s: %s", url, e)
    return ""

In [12]:
def scrape_listing(session, page_cap=20):
    all_urls, seen, page = [], set(), 1
    next_url = build_listing_url(page=1)
    prev_set = set()

    while next_url and page <= page_cap:
        html, _ = fetch_html(session, next_url)
        product_urls, hinted_next = parse_listing(html)
        logging.info("Página %d | %d produtos", page, len(product_urls))

        if not product_urls:
            logging.info("Sem produtos. Encerrando.")
            break

        cur_set = set(product_urls)
        if cur_set == prev_set:
            logging.info("Página repetida. Encerrando.")
            break
        prev_set = cur_set

        added = 0
        for u in product_urls:
            if u not in seen:
                seen.add(u); all_urls.append(u); added += 1
        if added == 0:
            logging.info("Nenhum novo produto. Encerrando.")
            break

        page += 1
        next_url = hinted_next or build_listing_url(page=page)
    return all_urls

# ====== OUTPUTS ======
def write_outputs(products_data, out_csv, out_json):
    csv_columns = ["site", "categoria_textura", "nome", "subtitulo", "preco", 
                   "beneficios", "ingredientes", "tamanho", "tipos_de_pele", "imagem"]
    # CSV
    with open(out_csv, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=csv_columns)
        w.writeheader()
        for product in products_data:
            w.writerow({k: product.get(k, "") for k in csv_columns})

    # JSON
    json_data = [{k: product.get(k, "") for k in csv_columns} for product in products_data]
    with open(out_json, "w", encoding="utf-8") as f:
        json.dump(json_data, f, ensure_ascii=False, indent=2)

In [13]:
def run_scraper(out_csv="ollie_produtos.csv", out_json="ollie_produtos.json",
                images_dir="images/", max_retries=3, timeout=20, max_products=50):
    s = make_session(max_retries=max_retries, timeout=timeout)
    os.makedirs(os.path.dirname(out_csv) or ".", exist_ok=True)
    os.makedirs(os.path.dirname(out_json) or ".", exist_ok=True)

    product_urls = scrape_listing(s, page_cap=20)
    logging.info("Total de URLs: %d", len(product_urls))

    products_data = []
    seen_names = set()

    for i, url in enumerate(product_urls, 1):
        if len(products_data) >= max_products:
            logger.info("Limite de %d produtos atingido. Parando.", max_products)
            break

        html, _ = fetch_html(s, url)
        data = parse_product(html, url)
        if data is None:
            logging.info("Skip (exclusão): %s", url)
            continue

        nome = (data.get("nome") or "").strip().lower()
        if nome in seen_names:
            logging.info("Skip (duplicado): %s", nome)
            continue
        seen_names.add(nome)

        slug = slugify(data.get("nome") or os.path.basename(urlparse(url).path))
        download_image(s, data.get("_imagem_url",""), images_dir, slug)

        # limpar campos internos
        clean_data = {k: v for k, v in data.items() if not k.startswith("_")}
        products_data.append(clean_data)

        if i % 3 == 0:
            write_outputs(products_data, out_csv, out_json)
            logging.info("Parcial salva (%d itens).", len(products_data))

    write_outputs(products_data, out_csv, out_json)
    logging.info("Finalizado: %d itens", len(products_data))
    return len(products_data), out_csv, out_json, images_dir


In [14]:
if __name__ == "__main__":
    OUT_CSV = "ollie_produtos.csv"
    OUT_JSON = "ollie_produtos.json"
    IMAGES_DIR = "images/"
    MAX_RETRIES = 3
    TIMEOUT = 20
    MAX_PRODUCTS = 50

    try:
        n_final, csv_path, json_path, img_dir = run_scraper(
            out_csv=OUT_CSV,
            out_json=OUT_JSON,
            images_dir=IMAGES_DIR,
            max_retries=MAX_RETRIES,
            timeout=TIMEOUT,
            max_products=MAX_PRODUCTS,
        )
        print("\n=== SCRAPING CONCLUÍDO ===")
        print("Itens coletados:", n_final)
        print("CSV:", os.path.abspath(csv_path))
        print("JSON:", os.path.abspath(json_path))
        print("Imagens:", os.path.abspath(img_dir))
    except Exception as e:
        print("Erro durante o scraping:", e)


2025-09-22 19:48:53,380 | INFO | Página 1 | 33 produtos
2025-09-22 19:48:54,460 | INFO | Página 2 | 6 produtos
2025-09-22 19:48:56,146 | INFO | Página 3 | 33 produtos
2025-09-22 19:48:56,146 | INFO | Nenhum novo produto. Encerrando.
2025-09-22 19:48:56,147 | INFO | Total de URLs: 39
2025-09-22 19:48:58,662 | INFO | Skip (duplicado): glow hidratante facial fps50
2025-09-22 19:49:00,010 | INFO | Skip (duplicado): glow hidratante facial fps50
2025-09-22 19:49:01,212 | INFO | Skip (exclusão): https://meuollie.com.br/collections/loja-produtos-ollie/products/protetor-solar-hidratante-corporal-fps-60
2025-09-22 19:49:04,009 | INFO | Skip (duplicado): protetor solar em bastão com cor fps 95
2025-09-22 19:49:05,487 | INFO | Skip (duplicado): protetor solar em bastão com cor fps 95
2025-09-22 19:49:06,918 | INFO | Skip (duplicado): protetor solar em bastão com cor fps 95
2025-09-22 19:49:08,159 | INFO | Skip (duplicado): protetor solar em bastão com cor fps 95
2025-09-22 19:49:09,587 | INFO | Sk


=== SCRAPING CONCLUÍDO ===
Itens coletados: 14
CSV: /home/usuario/Área de trabalho/Dados/Ollie/ollie_produtos.csv
JSON: /home/usuario/Área de trabalho/Dados/Ollie/ollie_produtos.json
Imagens: /home/usuario/Área de trabalho/Dados/Ollie/images
